## 微博文档分类 

**数据集中的微博文档分为4类：体育、女性、文学、校园。训练集和测试集已经给定。**

In [1]:
import jieba
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

In [2]:
def get_contents_labels(dir_path, label):
    """
    获取dir_path下所有txt文件的分词结果以及对应的标签并返回
    :param dir_path: 比如“体育”，“文学”
    :param label:
    :return:
    """
    contents = []  # 存储dir_path下所有txt文件的分词
    labels = []
    os.chdir(dir_path)
    texts = os.listdir()
    for text in texts:
        with open(text) as f:
            content = f.read()
            content_cut = cut_text(content)
            contents.append(content_cut)
            f.close()
        labels.append(label)
    return contents, labels

In [3]:
def cut_text(content):
    """
    对content进行分词，返回分词组成的字符串，即"分词1+' '+分词2+' '+分词3+' '+..."
    :param content:
    :return:
    """
    content_jieba = jieba.cut(content)
    content_cut = ""
    for c in content_jieba:
        content_cut = content_cut + c + " "
    return content_cut

In [4]:
def get_train_features(contents, stop_words):
    """
    利用TfidfVectorizer得到所有文档的特征向量，即X_train
    :param contents:
    :param stop_words:
    :return:
    """
    tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5)
    x_train = tf.fit_transform(contents).toarray()  # 返回矩阵
    return x_train

In [5]:
def get_docs_labels(path):
    """
    返回path下所有txt文件的已分词contents和labels
    :param path:
    :return:
    """
    return_docs = []
    return_labels = []
    dirs = os.listdir(path=path)
    for dir_path in dirs:
        contents, labels = get_contents_labels(path + dir_path + "/", dir_path)
        return_docs += contents
        return_labels += labels
    return return_docs, return_labels

## 对训练集进行操作

In [6]:
train_path = "C:/Users/54243/text classification/train/"  # 训练集文件所在地址，这里使用绝对地址
stop_words = [word.strip() for word in open("C:/Users/54243/text classification/stop/"+"stopword.txt",
                                            encoding="utf-8").readlines()]  # 停用词列表

In [7]:
dirs = os.listdir(path=train_path)

In [8]:
dirs

['体育', '女性', '文学', '校园']

**可看出训练集的文档共有四个类别，将这些类别变成数字标签**

In [9]:
map_dict = {}
for i, j in enumerate(dirs):
    map_dict[j] = i  

**接下来获取训练集路径下所有文本，进行分词，返回分词后连接的文本以及对应的标签**

In [10]:
train_docs, train_labels = get_docs_labels(train_path)  

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\54243\AppData\Local\Temp\jieba.cache
Loading model cost 1.398 seconds.
Prefix dict has been built successfully.


In [11]:
train_docs

['球场 禁用 招数 “ 少先队员 之 惩戒 ” 冠 ',
 '可以 直接 到 编辑部 买 ， 地址 ， 北京 体育馆 路 8 号 ， 中国 体育 报业 总社 院内 ， 后楼 51700 ： 羽毛球 杂志 木有 支付宝 ， 木有 财付 通 ， 在 网上 订购 不 支持 货到付款 么 ？ 那么 北京 哪个 实体店 有 卖 12 月 的 《 羽毛球 》 杂志 ， 或者说 ， 能 去 你们 编辑 社买 不 ？ 地址 ？ 转发 ( 3 ) 评论 ( 4 ) 12 月 10 日 09 : 17 来自 新浪 微博 ',
 '直播 贴 阿内尔卡 正式 加盟 申花 ， 最 现场 连线 申花 新闻官 带来 第一手 消息 ， 上海 男篮 2 分 惜败 惨遭 4 连败 ， 北京 延续 不败 战绩 欧洲 足坛 激战 正酣 ， 米兰 被 弱旅 逼平 ， 拜仁 重夺 联赛 榜首 风云变幻 ， 霍华德 要求 离队 ， 湖人 退出 三方 交易 更 多 内容 请 听 11 : 00 的 《 990 体育新闻 》 934 : ',
 '组图 ： 金妍儿 黑丝 亮相 热心 公益   OL 套装 透 成熟   http : url . cn / 1bNQDP     ( 分享 自   腾讯 体育   ) $ LOTOzf $ ',
 '北京 23 分 落后 末节 大 反扑   惜败 佛山 终结 13 连胜     http : url . cn / 2nGdrq     ( 分享 自   腾讯 体育   ) $ LOTOzf $ ',
 '组图 ： 日本国 足 " 切腹 " 新 队服   遭讽 史 上 最 难看   http : url . cn / 13KtpQ     ( 分享 自   腾讯 体育   ) $ LOTOzf $ ',
 '科比 驾 240 万豪 车 上班   盼望 新座 驾 带来 新气象     http : url . cn / 1lZNIP     ( 分享 自   腾讯 体育   ) $ LOTOzf $ ',
 'CBA 诞生 本季 第 4 位 下课 主帅   佛山 官方 宣布 换龙汉杰   http : t . cn / SMTIyZ   （ 分享 自   新浪 体育 ）                 $ LOTOzf $ ',
 '皇马 新 C 罗 圆梦 终 亮相   夏

In [12]:
train_labels

['体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',
 '体育',

**计算TF-IDF向量**

In [13]:
tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5)
X_train = tf.fit_transform(train_docs).toarray()  # 将稀疏矩阵转换成普通的矩阵进行训练
y_train = [map_dict[label] for label in train_labels]

E:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [14]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09783047, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

**训练模型**

In [15]:
clf = MultinomialNB(alpha=0.001)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

## 对测试集操作

In [16]:
test_path = "C:/Users/54243/text classification/test/"
test_docs, test_labels = get_docs_labels(test_path)  # 得到分词后连接的文本，对应的标签
X_test = tf.transform(test_docs).toarray()  # 这里直接采用已经fit好的tf来转换测试集文本
y_test = [map_dict[label] for label in test_labels]

## 预测

In [17]:
print(np.mean(cross_val_score(clf, X_train, y_train, cv=10)))
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8853334817666532
0.91


**可以看出，在训练集上进行10折交叉验证得到的准确率为88.53%，对测试集进行预测得到的准确率为91%**

## 调参

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
clf_cv = GridSearchCV(estimator=MultinomialNB(), param_grid={"alpha": [0, 0.001, 0.003, 0.005, 0.007,0.009,
                                                                       0.01, 0.03, 0.05, 0.07, 0.09,
                                                                       0.1, 0.3, 0.5, 0.7, 0.9, 1.0]})
clf_cv.fit(X_train, y_train)
clf_cv.cv_results_

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
E:\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
E:\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
E:\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


{'mean_fit_time': array([0.50668804, 0.56398567, 0.50901977, 0.49802756, 0.56398567,
        0.49902582, 0.49869355, 0.58563995, 0.50202529, 0.54000147,
        0.54966283, 0.4973611 , 0.56065385, 0.49669321, 0.55399291,
        0.61129022, 0.5380017 ]),
 'std_fit_time': array([0.00941053, 0.09945741, 0.0089511 , 0.00339742, 0.09940413,
        0.0044944 , 0.00509466, 0.0726314 , 0.00740653, 0.04324771,
        0.04836344, 0.00471348, 0.09964829, 0.00294182, 0.02099915,
        0.11852884, 0.00329873]),
 'mean_score_time': array([0.06329314, 0.05996148, 0.05929621, 0.05996084, 0.06029439,
        0.05763181, 0.05929581, 0.07062181, 0.05929502, 0.0699544 ,
        0.06329179, 0.05929534, 0.0589629 , 0.05729802, 0.06495865,
        0.06362637, 0.05929502]),
 'std_score_time': array([0.00169858, 0.00163307, 0.00094212, 0.00163171, 0.00188582,
        0.00047058, 0.0037699 , 0.01296404, 0.00124692, 0.01282557,
        0.00618034, 0.00262186, 0.00215777, 0.00047098, 0.0077821 ,
        0.00

In [20]:
clf_cv.best_params_

{'alpha': 0.07}

**可以看出，最佳的参数为alpha=0.07**

In [21]:
y_pred = clf_cv.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.92


**调参后得到的最优分类器的预测准确率达到92%，比未调参前的准确率提升1%，这样的结果还是不错的。**